In [1]:
import json
import numpy as np
import numpy.linalg as la
import json
from sklearn import metrics, cross_validation
from sklearn.naive_bayes import GaussianNB, BernoulliNB
from sklearn.linear_model import LogisticRegression

In [2]:
def get_raw_data(filename):
    with open(filename) as file:
        data = json.load(file)
    return data

In [3]:
def get_feature_matrix(data, ingre_list, data_matrix):
    # go through each dish first; aka: row interation
    for i in range(len(data)):
        sample = data[i]
    #     sample_cui= sample['cuisine']
        sample_ing = sample['ingredients']
    # again, sample_ing is a list, have to go through each ingredient in the list
        for j in range(len(sample_ing)):
        # get the index of this ingredient
            if sample_ing[j] in ingre_list:
                data_matrix[i][ingre_list.index(sample_ing[j])] = 1
    return data_matrix


In [16]:
def get_accuracy_bernoulli(X, Y, train_indices, test_indices):
    BNB_model = BernoulliNB()
    X_train = X[train_indices]
    Y_train = Y[train_indices]
    X_test = X[test_indices]
    Y_test = Y[test_indices]
    BNB_model.fit(X_train, Y_train)
    Y_predicted = BNB_model.predict(X_test)
    count = 0
    for i in range(len(Y_predicted)):
        if Y_predicted[i] == Y_test[i]:
            count +=1
            
    return count/float(len(Y_test))

In [4]:
def get_accuracy_gaussian(X, Y, train_indices, test_indices):
    GNB_model = GaussianNB()
    X_train = X[train_indices]
    Y_train = Y[train_indices]
    X_test = X[test_indices]
    Y_test = Y[test_indices]
    GNB_model.fit(X_train, Y_train)
    Y_predicted = GNB_model.predict(X_test)
    count = 0
    for i in range(len(Y_predicted)):
        if Y_predicted[i] == Y_test[i]:
            count +=1
            
    return count/float(len(Y_test))

In [21]:
def get_accuracy_logistic(X, Y, train_indices, test_indices):
    LR_model = LogisticRegression()
    X_train = X[train_indices]
    Y_train = Y[train_indices]
    X_test = X[test_indices]
    Y_test = Y[test_indices]
    LR_model.fit(X_train, Y_train)
    Y_predicted = LR_model.predict(X_test)
    count = 0
    for i in range(len(Y_predicted)):
        if Y_predicted[i] == Y_test[i]:
            count +=1
            
    return count/float(len(Y_test))

In [6]:
# number of dishes
train_data = get_raw_data('./train.json')
test_data = get_raw_data('./test.json')

print ('# of dishes is ' + str(len(train_data)))
print ("test data length " + str(len(test_data)))

# of dishes is 39774
test data length 9944


In [7]:
# counting unique cuisine and ingredients
# also save unique ingredients to the ingre_list and save all the dishes name to train_labels
categ_hash = {}
ingre_hash = {}
ingre_list = []
train_labels = []
for i in range(len(train_data)):
    sample = train_data[i]
#     sample_id = sample['id']
    sample_cui= sample['cuisine']
    train_labels.append(sample_cui)
    sample_ing = sample['ingredients']
    # first time seen this cuisine, make the count =1
    if sample_cui not in categ_hash:
        categ_hash[sample_cui] = 1
    # otherwise count ++
    else:
        categ_hash[sample_cui] +=1
    # ingredients is a list
    for j in range(len(sample_ing)):
        if sample_ing[j] not in ingre_hash:
            ingre_hash[sample_ing[j]] = 1
            ingre_list.append(sample_ing[j])
        else:
            ingre_hash[sample_ing[j]] +=1

print('the # of distinct cuisine is ' + str(len(categ_hash)))
print('the # of distinct ingredients is ' + str(len(ingre_hash)))



the # of distinct cuisine is 20
the # of distinct ingredients is 6714


In [8]:
len(ingre_list)

6714

In [9]:
train_matrix = np.zeros(shape = ((len(train_data), len(ingre_hash))))

train_matrix = get_feature_matrix(train_data, ingre_list, train_matrix)
train_matrix.shape

(39774, 6714)

In [10]:
test_matrix = np.zeros(shape = ((len(test_data), len(ingre_hash))))
test_matrix = get_feature_matrix(test_data, ingre_list, test_matrix)
test_matrix.shape

(9944, 6714)

In [14]:
train_labels = np.array(train_labels)

In [15]:
for train_set, test_set in cross_validation.KFold(len(train_matrix), n_folds=3):
    accuracy = get_accuracy_gaussian(train_matrix, train_labels, train_set, test_set)
    print (accuracy)

0.37901644290239855
0.3829386031075577
0.37758334590435966


In [17]:
for train_set, test_set in cross_validation.KFold(len(train_matrix), n_folds=3):
    accuracy = get_accuracy_bernoulli(train_matrix, train_labels, train_set, test_set)
    print (accuracy)

0.684190677326897
0.6795142555438226
0.6869060190073918


In [22]:
for train_set, test_set in cross_validation.KFold(len(train_matrix), n_folds=3):
    accuracy = get_accuracy_logistic(train_matrix, train_labels, train_set, test_set)
    print (accuracy)

0.7758334590435964
0.7721375773118118
0.7786996530396741


In [27]:
# use logistic regression as the best classifier
LR_model = LogisticRegression()
LR_model.fit(train_matrix, train_labels)
predicted_labels = LR_model.predict(test_matrix)

In [25]:
# just checked the sample_submission, it needs output ID as well, which is....
test_id = []
for i in range(len(test_data)):
    sample = test_data[i]
    temp_id = sample['id']
    test_id.append(temp_id)

print (len(test_id))
print (test_id[23])

9944
11885


In [31]:
import csv

with open('dishes.csv', "w") as f:
    writer = csv.writer(f, delimiter=',')
    writer.writerow(['id', 'cuisine'])
    for index in range(len(test_id)):
        writer.writerow([test_id[index], predicted_labels[index]])